In [1]:
!pip install opendartreader

In [2]:
import OpenDartReader

# 1. API 키 설정
api_key = '57843872c61b2852422b4870c0a618d09ae430f7'
dart = OpenDartReader(api_key)

def get_issue_report_link(corp_name):
    # 2. 기업의 공시 목록 검색 
    # 최신 버전에서는 bgn_date 대신 start를 사용합니다.
    # p_nm 대신 kindness(공시종류)나 직접 검색어를 활용합니다.
    reports = dart.list(corp_name, start='2020-01-01')
    
    if reports is None or len(reports) == 0:
        return f"'{corp_name}'의 공시 기록을 찾을 수 없습니다."

    # 3. 보고서명에 '증권발행실적보고서'가 포함된 행만 필터링
    # 처음 사용하는 사용자도 필터링 과정을 이해하기 쉽게 명시적으로 작성했습니다.
    target_reports = reports[reports['report_nm'].str.contains('증권발행실적보고서', na=False)]
    
    if target_reports.empty:
        return f"'{corp_name}'의 '증권발행실적보고서'를 찾을 수 없습니다."

    # 4. 가장 최근 보고서 선택
    latest_report = target_reports.iloc[0]
    rcp_no = latest_report['rcept_no']
    report_nm = latest_report['report_nm']
    
    # 5. DART 웹사이트 연결 링크 생성
    url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}"
    
    return {
        "종목명": corp_name,
        "보고서명": report_nm,
        "접수번호": rcp_no,
        "DART링크": url
    }

# 실행 예시
result = get_issue_report_link('아셈스')
print(result)

{'종목명': '아셈스', '보고서명': '증권발행실적보고서', '접수번호': '20220127000385', 'DART링크': 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220127000385'}


In [3]:
!pip install finance-datareader

In [ ]:
import OpenDartReader
import FinanceDataReader as fdr
import pandas as pd
import time

# 1. 설정 (본인의 API 키 입력)
api_key = '57843872c61b2852422b4870c0a618d09ae430f7'
dart = OpenDartReader(api_key)

def get_all_reports():
    # 2. 거래소 상장 종목 리스트 가져오기 (약 2,700~2,800개)
    print("거래소 상장사 목록을 불러오는 중입니다...")
    df_krx = fdr.StockListing('KRX') 
    
    # 상장사 이름들만 리스트로 변환
    corp_names = df_krx['Name'].tolist()
    
    report_list = []
    total = len(corp_names)
    print(f"총 {total}개 기업을 대상으로 '증권발행실적보고서'를 검색합니다.")

    # 3. 모든 종목 순회하며 보고서 검색
    for idx, name in enumerate(corp_names):
        try:
            # 2020년부터 현재까지 공시 목록 조회
            reports = dart.list(name, start='2020-01-01')
            
            if reports is not None and not reports.empty:
                # '증권발행실적보고서'가 포함된 행만 필터링
                target = reports[reports['report_nm'].str.contains('증권발행실적보고서', na=False)]
                
                for _, report in target.iterrows():
                    report_list.append({
                        "종목명": name,
                        "보고서명": report['report_nm'],
                        "접수번호": report['rcept_no'],
                        "공시일자": report['submission_date'],
                        "DART링크": f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={report['rcept_no']}"
                    })
            
            # API 제한 방지 (초당 15건 미만 유지)
            time.sleep(0.12)
            
            if (idx + 1) % 50 == 0:
                print(f"진행 상황: {idx + 1}/{total} 완료 (현재 {len(report_list)}건 수집됨)")
                
        except Exception:
            # 에러(네트워크 등) 발생 시 덤덤하게 다음으로 넘어감
            continue

    # 4. 결과 저장
    final_df = pd.DataFrame(report_list)
    return final_df

# 실행 및 저장
all_market_reports = get_all_reports()
all_market_reports.to_excel("전체_증권발행실적보고서_리스트.xlsx", index=False)

print(f"\n✅ 수집 완료! 총 {len(all_market_reports)}개의 보고서 링크를 찾았습니다.")

거래소 상장사 목록을 불러오는 중입니다...
총 2901개 기업을 대상으로 '증권발행실적보고서'를 검색합니다.
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
진행 상황: 100/2901 완료 (현재 0건 수집됨)
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
진행 상황: 250/2901 완료 (현재 0건 수집됨)
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
진행 상황: 300/2901 완료 (현재 0건 수집됨)
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status":"013","message":"조회된 데이타가 없습니다."}
{"status"

In [ ]:
import OpenDartReader
import pandas as pd
import time

# 1. API 키 및 라이브러리 설정
api_key = '57843872c61b2852422b4870c0a618d09ae430f7'
dart = OpenDartReader(api_key)

def get_filtered_reports():
    print("🚀 수집 시작: 상장사 대상 증권발행실적보고서 추출")
    
    # [조건 1] stock_code가 있는 상장사 목록만 확보
    corps = dart.corp_codes
    target_corps = corps[corps['stock_code'].notnull()]
    
    report_list = []
    total = len(target_corps)
    
    for idx, row in target_corps.iterrows():
        try:
            # [조건 2] 2020년부터 현재까지 공시 목록 조회
            # 이름 불일치 방지를 위해 corp_code(고유번호)로 조회합니다.
            reports = dart.list(row['corp_code'], start='2020-01-01')
            
            if reports is not None and not reports.empty:
                # [조건 3] 보고서명에 '증권발행실적보고서'가 포함된 것만 추출
                target = reports[reports['report_nm'].str.contains('증권발행실적보고서', na=False)]
                
                for _, report in target.iterrows():
                    data = {
                        "종목명": row['corp_name'],
                        "종목코드": row['stock_code'],
                        "보고서명": report['report_nm'],
                        "접수번호": report['rcept_no'],
                        "공시일자": report['submission_date']
                    }
                    report_list.append(data)
                    print(f"📍 발견: {data['종목명']} ({data['공시일자']})")
            
            # API 호출 제한을 피하기 위한 최소한의 휴식
            time.sleep(0.12)
            
        except Exception:
            continue
            
    return pd.DataFrame(report_list)

# 실행 및 결과 확인
final_df = get_filtered_reports()
print(f"\n✅ 필터링 완료! 총 {len(final_df)}건의 보고서를 찾았습니다.")

# 수집된 리스트의 첫 5개 미리보기
print(final_df.head())